In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import griddata
import os

# ------------------ Utility ------------------
def add_noise(data, noise_level=0.0):
    return data * (1 + noise_level * np.random.randn(*data.shape))

# ------------------ Load Data ------------------
def load_data(mat_path, target_t=1.0):
    data = loadmat(mat_path)
    X_star = data['X_star']
    U_star = data['U_star']
    p_star = data['p_star']
    t_all = data['t'].flatten()
    t_idx = np.argmin(np.abs(t_all - target_t))
    x, y = X_star[:, 0], X_star[:, 1]
    u, v = U_star[:, 0], U_star[:, 1]
    p = p_star[:, t_idx].flatten()
    T = np.full_like(x, target_t)
    return np.column_stack((x, y, T)), np.column_stack((u, v, p)), x, y, p

# ------------------ Preprocess ------------------
def preprocess_data(X, Y):
    scaler = StandardScaler()
    Y_norm = scaler.fit_transform(Y)
    return Y_norm, scaler

# ------------------ Train Model ------------------
def train_mlp(X, Y_norm):
    model = MLPRegressor(hidden_layer_sizes=(60, 40), activation='relu', solver='adam',
                         max_iter=600, tol=1e-4, verbose=True, random_state=42)
    print(f"⚙️ Training on {X.shape[0]} points")
    model.fit(X, Y_norm)
    return model

# ------------------ Improved Velocity Plot ------------------
def plot_velocity_field(x, y, u, v, title):
    grid_size = 300
    x_lin = np.linspace(x.min(), x.max(), grid_size)
    y_lin = np.linspace(y.min(), y.max(), grid_size)
    Xg, Yg = np.meshgrid(x_lin, y_lin)
    pts = np.column_stack((x, y))

    # Interpolate onto regular grid
    Zu = griddata(pts, u, (Xg, Yg), method='cubic')
    Zv = griddata(pts, v, (Xg, Yg), method='cubic')
    mag = np.sqrt(Zu**2 + Zv**2)

    # Vorticity calculation
    dvdx, dvdy = np.gradient(Zv, x_lin, y_lin)
    dudx, dudy = np.gradient(Zu, x_lin, y_lin)
    vorticity = dvdx - dudy

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Velocity magnitude with improved colormap
    pcm1 = ax1.pcolormesh(Xg, Yg, mag, cmap='jet', shading='gouraud', vmin=0, vmax=1.2)
    ax1.streamplot(Xg, Yg, Zu, Zv, color='k', density=1.5, linewidth=0.7, arrowsize=1)
    ax1.add_patch(plt.Circle((0, 0), 0.5, color='gray'))
    ax1.set_title('Velocity Magnitude with Streamlines')
    fig.colorbar(pcm1, ax=ax1)

    # Vorticity with improved visualization
    pcm2 = ax2.pcolormesh(Xg, Yg, vorticity, cmap='RdBu_r', shading='gouraud', vmin=-3, vmax=3)
    ax2.add_patch(plt.Circle((0, 0), 0.5, color='gray'))
    ax2.set_title('Vorticity')
    fig.colorbar(pcm2, ax=ax2)

    for ax in [ax1, ax2]:
        ax.set_aspect('equal')
        ax.set_xlim(-2, 10)
        ax.set_ylim(-4, 4)

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    return fig

# ------------------ Pressure Plot ------------------
def plot_pressure(x, y, p_true, p_pred):
    grid_size = 200
    x_lin = np.linspace(x.min(), x.max(), grid_size)
    y_lin = np.linspace(y.min(), y.max(), grid_size)
    Xg, Yg = np.meshgrid(x_lin, y_lin)
    pts = np.column_stack((x, y))

    Z_pred = griddata(pts, p_pred, (Xg, Yg), method='cubic')
    Z_true = griddata(pts, p_true, (Xg, Yg), method='cubic')

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Predicted pressure
    pcm1 = ax1.pcolormesh(Xg, Yg, Z_pred, cmap='RdYlBu', shading='gouraud')
    ax1.add_patch(plt.Circle((0, 0), 0.5, color='gray'))
    ax1.set_title('Predicted Pressure')
    ax1.set_aspect('equal')
    fig.colorbar(pcm1, ax=ax1)

    # True pressure
    pcm2 = ax2.pcolormesh(Xg, Yg, Z_true, cmap='RdYlBu', shading='gouraud')
    ax2.add_patch(plt.Circle((0, 0), 0.5, color='gray'))
    ax2.set_title('True Pressure')
    ax2.set_aspect('equal')
    fig.colorbar(pcm2, ax=ax2)

    plt.tight_layout()
    return fig

# ------------------ Main ------------------
def main():
    # Create output directory on Desktop
    output_dir = os.path.join(os.path.expanduser('~'), 'Desktop', 'CFD_Results')
    os.makedirs(output_dir, exist_ok=True)

    mat_path = r'C:\Users\sauja\Downloads\cylinder_nektar_wake.mat'
    X, Y, x, y, p_true = load_data(mat_path, target_t=1.0)

    noise_level = 0.0
    Y_noisy = add_noise(Y, noise_level=noise_level)

    Y_norm, scaler = preprocess_data(X, Y_noisy)
    model = train_mlp(X, Y_norm)

    Y_pred = scaler.inverse_transform(model.predict(X))
    u_pred, v_pred, p_pred = Y_pred[:, 0], Y_pred[:, 1], Y_pred[:, 2]

    # Create and save figures
    fig1 = plot_pressure(x, y, p_true, p_pred)
    fig1.savefig(os.path.join(output_dir, 'pressure_comparison.png'),
                dpi=300, bbox_inches='tight')
    plt.close(fig1)

    fig2 = plot_velocity_field(x, y, u_pred, v_pred, "Cylinder Wake at t=1.0 (Predicted)")
    fig2.savefig(os.path.join(output_dir, 'velocity_field.png'),
                dpi=300, bbox_inches='tight')
    plt.close(fig2)

    print(f" Results saved to: {output_dir}")
    print(f"• Pressure comparison: {os.path.join(output_dir, 'pressure_comparison.png')}")
    print(f"• Velocity field: {os.path.join(output_dir, 'velocity_field.png')}")

if __name__ == "__main__":
    main()